## 모두를 위한 머신러닝/딥러닝 강의
김성훈 교수님의 모두를 위한 머신러닝/딥러닝 강의 중 lab 강의 코드입니다.
## Lab4_2 Loading Data from File

### With numpy

In [1]:
import sys, os
import tensorflow as tf
import numpy as np
os.listdir()

['.ipynb_checkpoints',
 'data-01-test-score.csv',
 'data-02-test-score.csv',
 'data-03-test-score.csv',
 'data-04-test-score.csv',
 'data-05-test-score.csv',
 'DLEL.zip',
 'DLEL_1_TensorFlow_Basics.ipynb',
 'DLEL_2_Linear_regression.ipynb',
 'DLEL_3_Minimizing_Cost.ipynb',
 'DLEL_4_2_Loading_Data_from_File.ipynb',
 'DLEL_4_Multi-variable_linear_regression.ipynb']

In [2]:
tf.set_random_seed(777) # for reproducibility
config = {'fname' : 'D:\dev\DLEL\data-01-test-score.csv', 'dtype' : np.float32, 'delimiter' : ','} 
xy = np.loadtxt(**config)
x_data = xy[:, 0:3]
y_data = xy[:, -1]
print(x_data.shape, y_data.shape)

(25, 3) (25,)


In [3]:
# Make sure the shape and data are OK
X = tf.placeholder(tf.float32, shape = [None, x_data.shape[1]])
Y = tf.placeholder(tf.float32, shape = [None])

W = tf.Variable(tf.random_normal([3, 1]), name = 'weight')
b = tf.Variable(tf.random_normal([1]), name = 'bias')

# Hypothesis
hypothesis = tf.matmul(X, W) + b

# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# Minimize
opt = tf.train.GradientDescentOptimizer(learning_rate = 1e-5)
train = opt.minimize(cost)

# Launch the graph in a session
sess = tf.Session()

# Initializes global variables in the graph
sess.run(tf.global_variables_initializer())

for step in range(2001):
    cost_val, hy_val, _ = sess.run([cost, hypothesis, train], feed_dict = {X : x_data, Y : y_data})
    if step % 1000 == 0:
        print(step, 'cost = {}'.format(cost_val), '\nPrediction\n{}'.format(hy_val))

0 cost = 21178.71875 
Prediction
[[ 22.04806328]
 [ 21.61978722]
 [ 24.09669304]
 [ 22.29300499]
 [ 18.6339016 ]
 [  7.2669735 ]
 [ 12.33102989]
 [  3.15051103]
 [ 14.34794426]
 [  4.25342369]
 [ 14.48570824]
 [ 10.67806816]
 [ 28.80463982]
 [ 29.29880333]
 [ 11.23783684]
 [ 18.6465435 ]
 [ 31.18945122]
 [ 13.34466362]
 [ 28.84174156]
 [ 25.66280937]
 [ 15.08476067]
 [ 16.79836845]
 [ 15.92455101]
 [ 31.36112022]
 [ 24.98636436]]
1000 cost = 1118.75341796875 
Prediction
[[ 152.37536621]
 [ 181.65196228]
 [ 179.85574341]
 [ 193.99241638]
 [ 139.83654785]
 [ 100.23703003]
 [ 142.99723816]
 [ 102.90788269]
 [ 167.2350769 ]
 [ 150.75358582]
 [ 139.2648468 ]
 [ 136.01049805]
 [ 187.65908813]
 [ 158.62748718]
 [ 143.24215698]
 [ 182.70162964]
 [ 152.90055847]
 [ 170.66418457]
 [ 179.43598938]
 [ 160.4236145 ]
 [ 168.05151367]
 [ 168.73977661]
 [ 160.88987732]
 [ 157.95100403]
 [ 189.32693481]]
2000 cost = 1086.204345703125 
Prediction
[[ 150.84555054]
 [ 181.65614319]
 [ 178.79142761]
 [ 194

In [4]:
# Ask my score
print('Your score will be ', sess.run(hypothesis, feed_dict = {X : [[100, 70, 101]]}))

Your score will be  [[ 179.80020142]]


### Queue Runners 

어차피 학습은 보통 mini-batch 단위로 이루어지므로 학습하고자하는 모든 데이터를 메모리 상에 올릴 필요가 없고 mini-batch 단위로 올려도 무방하다. Tensorflow에서는 이와 같은 기능을 수행하는 Queue-runner라는 것을 구현해놓았다. 기능은 아래의 그림과 같다.  
참고 : https://tensorflowkorea.gitbooks.io/tensorflow-kr/content/g3doc/how_tos/threading_and_queues/
  
![alt text](http://i.imgur.com/dbjgw1E.png)  
예제는 아래의 기능을 수행하는 코드를 짜본다.
![alt text](http://i.imgur.com/SZXiKNW.png)

In [5]:
tf.set_random_seed(777) # for reproducibility
print(os.listdir())
data_list = os.listdir()
data_list = list(filter(lambda x : x.startswith('data'), data_list))
data_list # 5개의 데이터로 queue를 이용한 learning을 해보자

['.ipynb_checkpoints', 'data-01-test-score.csv', 'data-02-test-score.csv', 'data-03-test-score.csv', 'data-04-test-score.csv', 'data-05-test-score.csv', 'DLEL.zip', 'DLEL_1_TensorFlow_Basics.ipynb', 'DLEL_2_Linear_regression.ipynb', 'DLEL_3_Minimizing_Cost.ipynb', 'DLEL_4_2_Loading_Data_from_File.ipynb', 'DLEL_4_Multi-variable_linear_regression.ipynb']


['data-01-test-score.csv',
 'data-02-test-score.csv',
 'data-03-test-score.csv',
 'data-04-test-score.csv',
 'data-05-test-score.csv']

In [6]:
filename_queue = tf.train.string_input_producer(data_list, shuffle=True, name='filename_queue')

reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

# Default values, in case of empty columns. Also specifies the type of the
# decoded result.
record_defaults = [[0.], [0.], [0.], [0.]]
xy = tf.decode_csv(value, record_defaults=record_defaults)

# collect batches of csv in
train_x_batch, train_y_batch = tf.train.batch([xy[0:-1], xy[-1:]], batch_size = 5)

# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 3])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([3, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# Hypothesis
hypothesis = tf.matmul(X, W) + b

# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

# Start populating the filename queue.
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

for step in range(2001):
    x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
    cost_val, hy_val, _ = sess.run(
        [cost, hypothesis, train], feed_dict={X: x_batch, Y: y_batch})
    if step % 1000 == 0:
        print(step, "Cost: ", cost_val, "\nPrediction:\n", hy_val)

coord.request_stop()
coord.join(threads)

# Ask my score
print("Your score will be ",
      sess.run(hypothesis, feed_dict={X: [[100, 70, 101]]}))

print("Other scores will be ",
      sess.run(hypothesis, feed_dict={X: [[60, 70, 110], [90, 100, 80]]}))

0 Cost:  105405.0 
Prediction:
 [[-131.55688477]
 [-165.2099762 ]
 [-159.19018555]
 [-172.29362488]
 [-128.71299744]]
1000 Cost:  13.9185 
Prediction:
 [[ 156.16769409]
 [ 182.81266785]
 [ 182.57078552]
 [ 200.10159302]
 [ 137.10035706]]
2000 Cost:  10.7483 
Prediction:
 [[ 155.48178101]
 [ 183.48394775]
 [ 182.45397949]
 [ 200.06747437]
 [ 137.90686035]]
Your score will be  [[ 177.21903992]]
Other scores will be  [[ 186.71313477]
 [ 178.01908875]]
